In [235]:
# Purpose:  Model topics for group 1 ( 1st democratic debate)
# Author:  Preeti Javaji
# Date:  05/15/16
# Course:  MAS DSE capstone, Spring 2016

In [268]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import math

pd.set_option('display.max_colwidth', 240)
pd.set_option('display.max_columns', 50)

import sys
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB


print 'done'

done


In [269]:
# Function defs.

# return string with non-ascii chars removed
def to_ascii(s):
    returnstr = s.strip()
    returnstr = "".join([ch for ch in returnstr if ord(ch)< 128])
    return returnstr

print 'done'

done


In [270]:
# Load data file & clean the 'text' column.

df = pd.read_csv("Sentiment_backup.csv")

# Convert to ascii
df['ascii'] = df['text'].apply(to_ascii)
#df['ascii'] = df['ascii'].str.encode('ascii',errors='ignore') 
df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
df['ascii'] = df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
df['ascii'] = df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
df['ascii'] = df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
df['ascii'] = df['ascii'].str.lower()

df['K_sentiment'] = df['sentiment']
print df.shape
#print df[df['ascii']!=df['text']][['ascii', 'K_sentiment']].head(10)
df[['text', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

(4333, 23)


,text,ascii
0,RT @TheBaxterBean: Scott Walker's Abortion Ban Allows Rapist Father To Sue For Emotional Distress http://t.co/rHMvgumuir #GOPDebate @Badger��_,rt @thebaxterbean scott walkers abortion ban allows rapist father to sue for emotional distress url #gopdebate @badger_
1,"RT @mydaughtersarmy: Dear GOP,\rIf you want to stop abortions... \r\r#GOPDebate \rhttp://t.co/RbL1t4iPx6","rt @mydaughtersarmy dear gop,\rif you want to stop abortions... \r\r#gopdebate \rurl"
2,RT @Bipartisanism: A message to the #GOPDebate candidates: http://t.co/WFK5CmVgiN,rt @bipartisanism a message to the #gopdebate candidates url


In [271]:
# Show the class frequencies.
counts = df['subject_matter'].value_counts()
total = counts.sum()

counts = counts.apply(lambda x: ("Count: %d" % x, "Frequency: %.2f" % (float(x)/total)) )

#print("%.2f" % a)
#string = 'string%d' % (i,)

print counts
print type(counts)

None of the above                       (Count: 1271, Frequency: 0.33)
Religion                                 (Count: 407, Frequency: 0.11)
Women's Issues (not abortion though)     (Count: 362, Frequency: 0.10)
Racial issues                            (Count: 353, Frequency: 0.09)
Abortion                                 (Count: 293, Frequency: 0.08)
Jobs and Economy                         (Count: 251, Frequency: 0.07)
Immigration                              (Count: 211, Frequency: 0.06)
FOX News or Moderators                   (Count: 188, Frequency: 0.05)
LGBT issues                              (Count: 126, Frequency: 0.03)
iran deal                                 (Count: 74, Frequency: 0.02)
Healthcare (including Medicare)           (Count: 67, Frequency: 0.02)
Gun Control                               (Count: 61, Frequency: 0.02)
Foreign Policy                            (Count: 41, Frequency: 0.01)
ISIS                                      (Count: 40, Frequency: 0.01)
none  

In [272]:
# Add the 1st democratic debate transcript
D1_df = pd.read_csv("labeled_transcripts/D1_proc_labeled.csv")
D1_df.head()

,Unnamed: 0,duration,speaker,speaker_type,start_time,text,topic
0,0,311,NaN,NaN,0,"ANDERSON COOPER, CNN ANCHOR: And good evening everyone. We are live at the Wynn Resort in Las Vegas for the CNN/Facebook Democratic Presidential Debate. Welcome. The five major candidates are about to face off for the first time in a...",intro
1,1,50,CHEERING,audience,1255,[cheering],intro
2,2,50,APPLAUSE,audience,1297,(APPLAUSE),intro
3,3,50,APPLAUSE,audience,1340,(APPLAUSE),intro
4,4,50,APPLAUSE,audience,1375,(APPLAUSE),intro


In [273]:
#clean it up too
D1_df['ascii'] = D1_df['text'].apply(to_ascii)

D1_df['ascii'] = D1_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D1_df['ascii'] = D1_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D1_df['ascii'] = D1_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D1_df['ascii'] = D1_df['ascii'].str.lower()


In [274]:
D1_df['topic'] = D1_df['topic'].str.lower()
a = D1_df['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'affirmative action',
 'benghazi',
 'bipartisanship',
 'black lives matter',
 'capitalism',
 'climate change',
 'clintons emails',
 'closing',
 'college tuition',
 'domestic surveillance',
 'environment',
 'family leave',
 'flipflopping',
 'gun control',
 'immigration reform',
 'income inequality',
 'insiders vs outsiders',
 'intro',
 'judgment and experience',
 'legalizing marijuana',
 'me vs obama',
 'military intervention',
 'national security',
 'none',
 'police violence',
 'progressivism',
 'russia in syria',
 'social security',
 'socialism',
 'syria',
 'va healthcare',
 'wall street']

In [275]:
# Now read hashtag tweets
D2_df = pd.read_csv("labeled_transcripts/HashtagTweets_updated.csv")
D2_df.head()

,topic,tweetid,text
0,benghazi,7.050000e+17,"""We will never forget #Benghazi @HillaryClinton https://t.co/lkfUGTaca4 #veterans #honor #courage #commitment #SuperTuesday #VoteTrump """
1,benghazi,7.050000e+17,"""##Benghazi Heroes Endorse Donald Trump https://t.co/v6jUWZdy0m """
2,benghazi,7.050000e+17,""".@hillaryclinton Great shot! Was this before or after the imaginary snipers? #VinceFoster #Benghazi #Whitewater https://t.co/0WnW2z4GAm """
3,benghazi,7.050000e+17,"""(IBD) Lying About #Benghazi Terrorist Attack Sole Hillary Accomplishment - https://t.co/ncx9a5LEmk #PJNET https://t.co/3Z7RV3ydv5 123 """
4,benghazi,7.050000e+17,"""what about the parents of #benghazi victims you lied to? @HillaryClinton @sitflyr """


In [276]:
#clean it up
#clean it up too
D2_df['ascii'] = D2_df['text'].apply(to_ascii)

D2_df['ascii'] = D2_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
D2_df['ascii'] = D2_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")

# Remove eol symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbols
D2_df['ascii'] = D2_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
D2_df['ascii'] = D2_df['ascii'].str.lower()

In [277]:
# Now take the text and topic from both the dataframes and concat
D1_new = D1_df[['ascii','topic']]
T_DF = df[['ascii','subject_matter']]

T_DF = T_DF.rename(columns = {'subject_matter':'topic'})

In [278]:
D2_new = D2_df[['ascii','topic']]
T_new1 = pd.concat([T_DF,D1_new])
print T_new1.shape
T_new = pd.concat([T_new1,D2_new])
print T_new.shape

(4863, 2)
(6382, 2)


In [279]:
# clean the topics before encoding
# first convert to lower case
T_new['topic'] = T_new['topic'].str.lower()
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

[nan,
 'abortion',
 'affirmative action',
 'benghazi',
 'billclinton',
 'bipartisanship',
 'black lives matter',
 'border security',
 'bordersecurity',
 'capitalism',
 'climate change',
 'clinton emails',
 'clintons emails',
 'closing',
 'cnn',
 'cnndebate',
 'cnntownhall',
 'college',
 'college tuition',
 'common core',
 'domestic surveillance',
 'environment',
 'family leave',
 'familyleave',
 'flintwater',
 'flintwatercrisis',
 'flipflopping',
 'foreign aid',
 'foreign policy',
 'fox news or moderators',
 'gun control',
 'healthcare (including medicare)',
 'homelandsecurity',
 'immigration',
 'immigration reform',
 'income inequality',
 'insiders vs outsiders',
 'intro',
 'iran',
 'iran deal',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'judgment and experience',
 'legalizing marijuana',
 'lgbt issues',
 'marijuana',
 'me vs obama',
 'military intervention',
 'national security',
 'none',
 'none of the above',
 'obama',
 'obamacare',
 'police violence',
 'progressivism',
 'ra

# After seeing the list above we will combine:
 1. 'black lives matter', 'affirmative action', 'racial issues',  'police violence'
 2. <removed >
 3. 'climate change', 'environment',
 4. 'clinton emails', 'clintons emails',
 5. 'cnn', 'cnndebate',  'cnntownhall',
 6. 'college', 'college tuition',
 7. 'flintwater', 'flintwatercrisis',
 8. 'foreign policy',  'foreign aid', 'military intervention', 'homelandsecurity', 'domestic surveillance','national security',
 9. 'healthcare (including medicare)', 'obamacare',
 10. 'immigration',  'immigration reform','border security',  'bordersecurity',
 11. 'iran',  'iran deal',
 12. 'isis',  'radical islam',
 13. 'jobs and economy', 'wall street', 'income inequality', 'taxes',
 14. 'legalizing marijuana',  'marijuana',
 15. 'me vs obama',  'obama',
 16. 'none', 'none of the above',
 17. 'syria',  'russia in syria',
 18. 'va healthcare', 'veterans',
 19. 'zika',  'zikavirus'

In [289]:
#1
T_new.ix[T_new['topic'].isin( ['racial issues','police violence','affirmative action']),'topic'] = 'black lives matter'
#2 #3
T_new.ix[T_new['topic'] == 'climate change','topic'] = 'environment'
#4
T_new.ix[T_new['topic'] == 'clintons emails','topic'] = 'clinton emails'
#5
T_new.ix[T_new['topic'].isin( ['cnndebate','cnntownhall']),'topic'] = 'cnn'
#6
T_new.ix[T_new['topic'] == 'college tuition','topic'] = 'college'
#7
T_new.ix[T_new['topic'] == 'flintwatercrisis','topic'] = 'flintwater'
#8
T_new.ix[T_new['topic'].isin(['foreign aid', 'military intervention', 'homelandsecurity',
                             'domestic surveillance','national security']),'topic'] = 'foreign policy'
#9
T_new.ix[T_new['topic'] == 'obamacare','topic'] = 'healthcare (including medicare)'
#10
T_new.ix[T_new['topic'].isin( ['bordersecurity','immigration reform','border security']),'topic'] = 'immigration'
#11
T_new.ix[T_new['topic'] == 'iran deal','topic'] = 'iran'
#12
T_new.ix[T_new['topic'] == 'radical islam','topic'] = 'isis'
#13
T_new.ix[T_new['topic'].isin( ['wall street', 'income inequality', 'taxes']),'topic'] = 'jobs and economy'
#14
T_new.ix[T_new['topic'] == 'legalizing marijuana','topic'] = 'marijuana'
#15
T_new.ix[T_new['topic'] == 'me vs obama','topic'] = 'obama'
#16
T_new.ix[T_new['topic'] == 'none of the above','topic'] = 'none'
#17
T_new.ix[T_new['topic'] == 'russia in syria','topic'] = 'syria'
#18
T_new.ix[T_new['topic'] == 'va healthcare','topic'] = 'veterans'
#19
T_new.ix[T_new['topic'] == 'zikavirus','topic'] = 'zika'
#20
T_new.ix[T_new['topic'] == 'familyleave','topic'] = 'family leave'


In [290]:
a = T_new['topic'].unique()
topics = list(a)
topics.sort()
topics

['abortion',
 'benghazi',
 'billclinton',
 'bipartisanship',
 'black lives matter',
 'capitalism',
 'clinton emails',
 'cnn',
 'college',
 'common core',
 'environment',
 'family leave',
 'flintwater',
 'foreign policy',
 'fox news or moderators',
 'gun control',
 'healthcare (including medicare)',
 'immigration',
 'insiders vs outsiders',
 'iran',
 'iraq',
 'isis',
 'israel',
 'jobs and economy',
 'judgment and experience',
 'lgbt issues',
 'marijuana',
 'none',
 'obama',
 'progressivism',
 'religion',
 'social security',
 'socialism',
 'syria',
 'torture',
 'veterans',
 "women's issues (not abortion though)",
 'zika']

In [291]:
# remove intro closing nan and flipflopping 

T_new =T_new[~T_new['topic'].isin(['intro','flipflopping','closing'])]
T_new.shape

(6312, 2)

In [292]:
# set all nans in topic to none
T_new.ix[T_new['topic'].isnull(),'topic'] = 'none'

In [293]:
T_new['topic'].value_counts()

none                                    2073
religion                                 407
black lives matter                       369
women's issues (not abortion though)     362
abortion                                 293
jobs and economy                         275
immigration                              230
cnn                                      228
fox news or moderators                   188
billclinton                              180
socialism                                141
iran                                     132
healthcare (including medicare)          132
lgbt issues                              126
gun control                              123
flintwater                               102
isis                                      97
benghazi                                  92
israel                                    92
foreign policy                            88
syria                                     85
veterans                                  75
obama     

In [294]:
# Sample the data so as to get equal numbers of each class.
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(T_new['ascii'],T_new['topic'], test_size=0.3, random_state=42)


In [295]:
    
print len(X_train), len(X_test) , len(y_train) , len(y_test)

4418 1894 4418 1894


In [297]:
stop_words = pd.read_table('/Users/preetijavaji/Documents/DSE/capstone/stop_words.org',names='words')
stop_words_list = list(stop_words['w'])

In [298]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer   =  CountVectorizer(min_df = 1,stop_words = stop_words_list, ngram_range = (1,2))
X_f_train = vectorizer.fit_transform(X_train)

In [299]:
classifiers = []
#classifiers.append( svm.SVC() )
classifiers.append( OneVsOneClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(LinearSVC(random_state=0)) )
classifiers.append( OneVsRestClassifier(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)) )

#classifier_1 = OneVsOneClassifier(svm.SVC())
#OneVsOneClassifier(LinearSVC(random_state=0)).fit(X, y).predict(X)

cv = 10  # Number of crossvalidation folds to use.
scores = []

for classifier in classifiers:
    scores.append( cross_validation.cross_val_score(classifier, X_f_train, y_train, cv=cv) )
    
for score in scores:
    print score
    print score.mean()
    print

print 'done'

[ 0.7673913   0.75604396  0.79249448  0.80846325  0.77303371  0.78587699
  0.78341014  0.774942    0.78688525  0.8       ]
0.782854107384

[ 0.79130435  0.77582418  0.78366446  0.81737194  0.7752809   0.79498861
  0.79262673  0.78190255  0.81967213  0.82352941]
0.795616525303

[ 0.74347826  0.72747253  0.75496689  0.77282851  0.73258427  0.7357631
  0.75576037  0.76334107  0.7587822   0.76470588]
0.750968307087

done


In [300]:
XT_f = vectorizer.transform(X_test)
predicteds = []
for classifier in classifiers:
    classifier.fit(X_f_train, y_train)
    yy = classifier.predict(XT_f)
    print yy.shape
    predicteds.append( classifier.predict(XT_f))


(1894,)
(1894,)
(1894,)


In [301]:
# Apply classifiers to the whole (not sampled) dataset and look at the confusion matrix and classification report for each.

# Format the whole dataset.
X_all = T_new['ascii'].values
y_all = T_new['topic'].values


# Create feature vectors from the tweets. Use the same feature vector as above.
X_featurized_all = vectorizer.transform(X_all)

# Generate predictions using the classifier trained on the equally sampled data.
predicteds = []
for classifier in classifiers:
    predicteds.append( classifier.predict(X_featurized_all) )
    
for y_pred_all in predicteds:
    print accuracy_score(y_all, y_pred_all)
    #print '   Neg ', 'Neu ', 'Pos'
    print confusion_matrix(y_all, y_pred_all)
    print classification_report(y_all, y_pred_all)
    print


0.921419518378
[[252   0   0 ...,   0  15   0]
 [  0  90   0 ...,   1   0   0]
 [  0   0 175 ...,   0   0   0]
 ..., 
 [  0   0   0 ...,  73   0   0]
 [  5   0   0 ...,   0 320   0]
 [  0   0   0 ...,   0   0  30]]
             precision    recall  f1-score   support

   abortion       0.95      0.86      0.90       293
   benghazi       0.98      0.98      0.98        92
billclinton       0.99      0.97      0.98       180
bipartisanship       1.00      0.50      0.67         2
black lives matter       0.95      0.90      0.92       369
 capitalism       1.00      0.60      0.75         5
clinton emails       1.00      0.84      0.92        45
        cnn       0.95      0.97      0.96       228
    college       1.00      0.85      0.92        27
common core       1.00      1.00      1.00         1
environment       1.00      0.87      0.93        39
family leave       1.00      0.81      0.90        32
 flintwater       1.00      0.99      1.00       102
foreign policy       0.98   

In [302]:
# read the tweet sample into a df
Tweet_df = pd.read_table('tweet_g1.csv',sep=',',names=['tweetid','tweet','c3','c4','c5','c6','c7','c8'])
Tweet_df

,tweetid,tweet,c3,c4,c5,c6,c7,c8
0,6.533330e+17,"""#HUNT: ODDS #STILL FAVOR #HILLARY... http://t.co/PJb7f1baGv """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:13:51 +0000 2015,488
1,6.533340e+17,"""RT @Drudge_Report_: #HUNT: ODDS #STILL FAVOR #HILLARY... http://t.co/PJb7f1baGv """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:18:30 +0000 2015,0
2,6.381690e+17,"""Can't believe Donald Trump winning the election http://t.co/H3XhKDnlOi """,Tweet_2015_10_11,NaN,NaN,1.440990e+12,Mon Aug 31 01:57:10 +0000 2015,16051
3,6.533340e+17,"""RT @DooleyFunnyAf: Can't believe Donald Trump winning the election http://t.co/H3XhKDnlOi """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:18:30 +0000 2015,0
4,6.533340e+17,"""GLA Article link http://t.co/qdV7YoDILp http://t.co/06oaRP4Q0n """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:18:30 +0000 2015,0
5,6.533330e+17,"""Oh my dear God ... #Villagerswithmajormancrushes https://t.co/JzUbMHegjg """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:14:14 +0000 2015,6
6,6.533340e+17,"""RT @digby56: Oh my dear God ... #Villagerswithmajormancrushes https://t.co/JzUbMHegjg """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:18:30 +0000 2015,0
7,6.533300e+17,"""Sanders: ���No�۪ I�۪m Not A Capitalist, ���I�۪m a Democratic Socialist�۪ - http://t.co/LrAAeprKBj via @TeaPartyOrg """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:02:50 +0000 2015,4
8,6.533340e+17,"""RT @ARnews1936: Sanders: ���No�۪ I�۪m Not A Capitalist, ���I�۪m a Democratic Socialist�۪ - http://t.co/LrAAeprKBj via @TeaPartyOrg """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:18:31 +0000 2015,0
9,6.533310e+17,"""Former President George H.W. Bush throws out the ceremonial first pitch to Jed Lowrie of the @astros. @GettySport http://t.co/jdwtmJJBSr """,Tweet_2015_10_11,NaN,NaN,1.444600e+12,Sun Oct 11 22:06:56 +0000 2015,30


In [303]:
# Convert to ascii
Tweet_df['ascii'] = Tweet_df['tweet'].apply(to_ascii)

# Remove hashtags
#df['ascii'] = df['ascii'].str.replace(r"#([A-Za-z0-9_]+)", " ")

# Remove handles
#df['ascii'] = df['ascii'].str.replace(r"@([A-Za-z0-9_]+)", " ")

# Remove URLs
#df['ascii'] = df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", " ")
# Replace URLs with "URL"
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"http([A-Za-z0-9_:.\/]+)", "URL")

# Remove punctuation symbols - but not @ or #
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"(['';:%()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"(['';:@%#()\+\*\"\…\“\”])", "")
#df['ascii'] = df['ascii'].str.replace(r"([;:@%#()\+\*\"\…\“\”])", "")

# Remove eol symbols
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"\n", " ")

# Remove &x symbolst
Tweet_df['ascii'] = Tweet_df['ascii'].str.replace(r"&[a-z]+", " ")

# Convert to lowercase
Tweet_df['ascii'] = Tweet_df['ascii'].str.lower()


Tweet_df[['tweet', 'ascii']].head(3)
#df[df['ascii'].str.contains("\n")][['text', 'ascii']].head(10)

,tweet,ascii
0,"""#HUNT: ODDS #STILL FAVOR #HILLARY... http://t.co/PJb7f1baGv """,#hunt odds #still favor #hillary... url
1,"""RT @Drudge_Report_: #HUNT: ODDS #STILL FAVOR #HILLARY... http://t.co/PJb7f1baGv """,rt @drudge_report_ #hunt odds #still favor #hillary... url
2,"""Can't believe Donald Trump winning the election http://t.co/H3XhKDnlOi """,cant believe donald trump winning the election url


In [304]:
# copy classifier 0 as the model for g1 and test it in spark on a sample 
import pickle
# save model in a pickle file
output = open('mymodel_g1.pkl', 'wb')
pickle.dump(classifiers[0],output)
output.close()

In [305]:
vec_file = open('myvec_g1.pkl','wb')
pickle.dump(vectorizer,vec_file)
vec_file.close()

In [306]:
X = Tweet_df['ascii'].values

#train_vectors = vectorizer.fit_transform(train_data)
#test_vectors = vectorizer.transform(test_data)
X_featurized = vectorizer.transform(X)
        
print X_featurized.shape
print 'done'

(10000, 35511)
done


In [314]:
predicted_new = []
for classifier in classifiers:
    predicted_new.append( classifier.predict(X_featurized) )

In [308]:
predicted_new[0]

array(['none', 'none', 'none', ..., 'none', 'none', 'none'], dtype=object)

In [309]:
print len(predicted_new[0]) , Tweet_df.shape

10000 (10000, 9)


In [315]:
Tweet_df['topic'] = predicted_new[0]

In [316]:
from collections import Counter

Keys = Counter(predicted_new[0]).keys() # equals to list(set(words))
values =  Counter(predicted_new[0]).values() # counts the elements' frequency
for k,v in zip(Keys,values):
    print k,':',v

fox news or moderators : 3
marijuana : 6
lgbt issues : 5
healthcare (including medicare) : 84
environment : 5
religion : 52
clinton emails : 1
cnn : 65
billclinton : 237
veterans : 10
social security : 7
benghazi : 150
israel : 11
iran : 5
immigration : 24
iraq : 10
black lives matter : 89
obama : 131
none : 8528
isis : 35
gun control : 237
jobs and economy : 18
abortion : 81
socialism : 123
syria : 14
women's issues (not abortion though) : 69


In [312]:
print len(Counter(predicted_new[0]).keys())

26


In [313]:
Tweet_df[Tweet_df['topic'] == "gun control"]['tweet']

21                 "@Wendy61342557 @nhdogmom @angloshok @SenSanders also, cars, knives, etc have different primary purposes. Only thing a gun is meant to do? "
90                                                     "#DonaldTrump:   ���Sometimes�۪ I carry a gun  https://t.co/E89TMOCKHE #MakeAmericaGreatAgain #2A #NRA "
108                    "@GottaLaff @MoneyForLies @crooksandliars when will GOP recognize that majority of Amns want stricter gun laws! http://t.co/duMS3iNEDE "
109           "RT @UdnSpeak4me: @GottaLaff @MoneyForLies @crooksandliars when will GOP recognize that majority of Amns want stricter gun laws! http://t.co��_ "
158                   "Bernie Sanders: Gun Manufacturers May Need to be Held Liable for Gun Crime http://t.co/vtA3zjJ4fA #tcot #lnyhbt http://t.co/p7LKnxT5Wv "
159           "RT @keeplibertyorg: Bernie Sanders: Gun Manufacturers May Need to be Held Liable for Gun Crime http://t.co/vtA3zjJ4fA #tcot #lnyhbt http://��_ "
197              "1.Ban "gun free zones"

In [173]:
Tweet_df[Tweet_df['topic'] == 'flint water crisis']['tweet']

Series([], Name: tweet, dtype: object)